In [1]:
import ee
from datetime import datetime

ee.Authenticate()
ee.Initialize()

Opening in existing browser session.

Successfully saved authorization token.


In [45]:
LAT = 57.2948 
LON = 22.5929 
AOI = ee.Geometry.Point(LAT, LON)
START_DATE = '2022-01-01'
END_DATE = '2023-01-01'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [42]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

In [48]:
dataset = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

# filter according to time interval
filtered_dataset = dataset.filterDate(ee.Date(START_DATE), ee.Date(END_DATE))


In [55]:
import numpy as np
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import torch
import torchvision.transforms as transforms
import cv2
import tempfile

df = pd.DataFrame()
data_list = []

point = ee.Geometry.Point(LAT, LON)

# filter
filtered_dataset_ = filtered_dataset.filterBounds(point)

for image in filtered_dataset_.toList(filtered_dataset_.size()).getInfo():
  image_date = datetime.utcfromtimestamp(image['properties']['system:time_start'] / 1000.0).strftime('%Y-%m-%d')
  print(f"Processing image for date: {image_date}")

  selected_image = ee.Image(image['id'])
  band_names = selected_image.bandNames()
  # print(band_names.getInfo())

  cloud_image = selected_image.select(['probability'])          
  # Use reduceRegion to get all values, then extract the specific band value
  cloud_dict = cloud_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=point, scale=10).getInfo()
  cloud = cloud_dict.get('probability')
  print(cloud)

  data_list.append({
                'date': image_date,
                'cloud_prob': cloud
                })
        
    
df = pd.DataFrame(data_list)

print(df)

Processing image for date: 2022-01-05
9
Processing image for date: 2022-01-05
9
Processing image for date: 2022-01-10
5
Processing image for date: 2022-01-10
6
Processing image for date: 2022-01-15
10
Processing image for date: 2022-01-15
10
Processing image for date: 2022-01-20
34
Processing image for date: 2022-01-20
35
Processing image for date: 2022-01-25
100
Processing image for date: 2022-01-25
100
Processing image for date: 2022-01-30
6
Processing image for date: 2022-01-30
6
Processing image for date: 2022-02-04
4
Processing image for date: 2022-02-04
4
Processing image for date: 2022-02-09
2
Processing image for date: 2022-02-09
2
Processing image for date: 2022-02-14
12
Processing image for date: 2022-02-14
12
Processing image for date: 2022-02-19
1
Processing image for date: 2022-02-19
1
Processing image for date: 2022-02-24
3
Processing image for date: 2022-02-24
3
Processing image for date: 2022-03-01
10
Processing image for date: 2022-03-01
10
Processing image for date: 2

In [ ]:
df.to_csv('data/cloud_prob_index6.csv')